In [6]:
import pandas as pd
import itertools
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

In [8]:
# Constants
grid_size = 5
spacing_xy = 48.8
spacing_z = 27.112
half_shift = spacing_xy / 4

# Board ID mapping
board_ids = {
    0: 'E1',   1: 'O10', 2: 'E2',  3: 'O2',  4: 'E3',
    5: 'O3',   6: 'E9',  7: 'O9',  8: 'E5',  9: 'O5',
    10: 'E6', 11: 'O6', 12: 'E7', 13: 'O7', 14: 'E8'
}

# CAEN-to-layer mapping
caen_mapping = {
    0: [(0, 24, 0), (25, 49, 1), (50, 62, '4a')],
    1: [(0, 24, 2), (25, 49, 3), (50, 61, '4b')],
    2: [(0, 24, 5), (25, 49, 6), (50, 62, '9a')],
    3: [(0, 24, 7), (25, 49, 8), (50, 61, '9b')],
    4: [(0, 24, 10), (25, 49, 11), (50, 62, '14a')],
    5: [(0, 24, 12), (25, 49, 13), (50, 61, '14b')],
    6: [(0, 24, 15), (25, 49, 16), (50, 62, '19a')],
    7: [(0, 24, 17), (25, 49, 18), (50, 61, '19b')],
    8: [(0, 24, 20), (25, 49, 21), (50, 62, '24a')]
}

records = []

for caen, ranges in caen_mapping.items():
    for start_ch, end_ch, layer in ranges:
        num_channels = end_ch - start_ch + 1

        if isinstance(layer, str) and layer.endswith('a'):
            position_offset = 0
            layer_number = int(layer[:-1])
        elif isinstance(layer, str) and layer.endswith('b'):
            position_offset = 13
            layer_number = int(layer[:-1])
        else:
            position_offset = 0
            layer_number = layer

        for i in range(num_channels):
            caen_channel = start_ch + i
            position = position_offset + i

            # Grid position
            row = position // grid_size
            col = position % grid_size

            # Centered x,y
            x = -(2 - col) * spacing_xy
            y = (row - 2) * spacing_xy

            # Apply shift for staggered layers
            if layer_number % 2 == 0:
                x += half_shift
                y += half_shift
            else:
                x -= half_shift
                y -= half_shift

            z = layer_number * spacing_z
            channel_num = caen_channel + 64 * caen
            #board_id = board_ids[layer_number]

            records.append({
                'CAEN_brd': caen,
                'CAEN_ch': caen_channel,
                'channel': channel_num,
                'layer': layer_number,
                'layer_ch': position,
                'x': x,
                'y': y,
                'z': z
                #'board_id': board_id
            })

# Create DataFrame
df = pd.DataFrame.from_records(records)

# Optional: sort by Ch# or any other field
df = df.sort_values(by='channel').reset_index(drop=True)

     CAEN_brd  CAEN_ch  channel  layer  layer_ch      x     y        z
0           0        0        0      0         0  -85.4 -85.4    0.000
1           0        1        1      0         1  -36.6 -85.4    0.000
2           0        2        2      0         2   12.2 -85.4    0.000
3           0        3        3      0         3   61.0 -85.4    0.000
4           0        4        4      0         4  109.8 -85.4    0.000
..        ...      ...      ...    ...       ...    ...   ...      ...
558         8       58      570     24         8   61.0 -36.6  650.688
559         8       59      571     24         9  109.8 -36.6  650.688
560         8       60      572     24        10  -85.4  12.2  650.688
561         8       61      573     24        11  -36.6  12.2  650.688
562         8       62      574     24        12   12.2  12.2  650.688

[563 rows x 8 columns]


In [10]:
# Step 1: Get all unique CAEN boards
caen_boards = df["CAEN_brd"].unique()

# Step 2: Generate all possible (CAEN_brd, CAEN_ch) combinations
full_index = pd.DataFrame([
    {"CAEN_brd": brd, "CAEN_ch": ch}
    for brd in caen_boards
    for ch in range(64)
])

# Step 3: Calculate 'channel' from CAEN_brd and CAEN_ch
full_index["channel"] = full_index["CAEN_brd"] * 64 + full_index["CAEN_ch"]

# Step 4: Merge with existing DataFrame
df_filled = pd.merge(
    full_index,
    df,
    on=["CAEN_brd", "CAEN_ch", "channel"],
    how="left",
    suffixes=('', '_orig')
)

# Step 5: Sort for readability
df_filled = df_filled.sort_values(by=["channel"]).reset_index(drop=True)

print(df_filled)


     CAEN_brd  CAEN_ch  channel  layer  layer_ch      x     y        z
0           0        0        0    0.0       0.0  -85.4 -85.4    0.000
1           0        1        1    0.0       1.0  -36.6 -85.4    0.000
2           0        2        2    0.0       2.0   12.2 -85.4    0.000
3           0        3        3    0.0       3.0   61.0 -85.4    0.000
4           0        4        4    0.0       4.0  109.8 -85.4    0.000
..        ...      ...      ...    ...       ...    ...   ...      ...
571         8       59      571   24.0       9.0  109.8 -36.6  650.688
572         8       60      572   24.0      10.0  -85.4  12.2  650.688
573         8       61      573   24.0      11.0  -36.6  12.2  650.688
574         8       62      574   24.0      12.0   12.2  12.2  650.688
575         8       63      575    NaN       NaN    NaN   NaN      NaN

[576 rows x 8 columns]


In [11]:
df.to_pickle("geometry_DF.pkl")